In [1]:
# 필요한 라이브러리 설치
!pip install pyxlsb

!pip install pandas openpyxl




In [2]:
import pandas as pd
import numpy as np 
import os


# 경로지정

In [3]:

# 파일 경로 지정
xlsb_file = r"C:\Users\work\Desktop\ori.xlsb" #원본
output_csv = r"C:\Users\work\Desktop\CH55, 56_Full cell LSV_1 mV_250122_01.csv" #수정

In [4]:
# 파일 존재 여부 확인
if not os.path.exists(xlsb_file):
    raise FileNotFoundError(f"파일이 존재하지 않습니다: {xlsb_file}")
else:
    print("파일 경로 확인 완료!")

# 1. xlsb 파일 읽기 (DCData1 시트, header=0으로 읽기)
df = pd.read_excel(xlsb_file, engine='pyxlsb', sheet_name="DCData1", header=0)

# 데이터프레임 열 개수 확인
print("데이터프레임 열 개수:", len(df.columns))

# 원본 데이터 열 목록 출력
print("원본 데이터 열 목록:", df.columns.tolist())

# 열 이름 수동 설정 (원본 데이터에 맞게 조정)
expected_columns = 19  # 원본 데이터에서 예상하는 열 개수
if len(df.columns) == expected_columns:
    df.columns = [
        "ListNo", "TestTime_s", "FileNo", "FileTime_s", "CycleNo", "CycleTime_s", "StepNo", "StepTime_s", 
        "_Current_A", "Voltage_V", "Power_W", "Load_Ohm", "SumQ_Ah", "AbsQ_Ah", "Temperature_C", 
        "Auxiliary1_V", "Auxiliary2_V", "Auxiliary3_V", "OCP_V"
    ]
else:
    raise ValueError(f"원본 데이터의 열 개수가 예상과 다릅니다. 현재 열 개수: {len(df.columns)}")

# 데이터프레임 열 확인 및 상위 데이터 출력
print("헤더 설정 후 데이터 열 목록:", df.columns.tolist())
print("데이터 샘플:\n", df.head())

파일 경로 확인 완료!
데이터프레임 열 개수: 19
원본 데이터 열 목록: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']
헤더 설정 후 데이터 열 목록: ['ListNo', 'TestTime_s', 'FileNo', 'FileTime_s', 'CycleNo', 'CycleTime_s', 'StepNo', 'StepTime_s', '_Current_A', 'Voltage_V', 'Power_W', 'Load_Ohm', 'SumQ_Ah', 'AbsQ_Ah', 'Temperature_C', 'Auxiliary1_V', 'Auxiliary2_V', 'Auxiliary3_V', 'OCP_V']
데이터 샘플:
    ListNo  TestTime_s  FileNo  FileTime_s  CycleNo  CycleTime_s  StepNo  \
0       1         NaN       1         NaN        1          NaN       1   
1       2         NaN       1         NaN        1          NaN       1   
2       3         NaN       1         NaN        1          NaN       1   
3       4         NaN       1         NaN        1          NaN       1   
4       5         NaN       1      

# 몇 행 쓸건지, 파라미터(전극 오차값(mV)) 값

In [5]:

# 2. 1801행부터 데이터를 선택
df = df.iloc[1799:]  # 1800행부터 선택 (인덱스는 0부터 시작하므로 1799)

# 파라미터 값 설정 (사용자가 변경할 값)
parameter = 5.9 # 초기값 설정, 필요에 따라 변경


In [6]:

# 필요한 열 선택 및 이름 변경
new_df = pd.DataFrame()
if "_Current_A" in df.columns and "Voltage_V" in df.columns:
    new_df["A(I열)"] = df["_Current_A"]  # 원본 A(I열)
    new_df["V(J열)"] = df["Voltage_V"]    # 원본 V(J열)
    
    # 새로운 열 추가
    new_df["mV(F.C Ag/AgCl)"] = df["Voltage_V"] * 1000  # V에서 mV로 변환
    new_df["mA/m2"] = (df["_Current_A"] * 1000) / 0.0007  # A에서 mA/m²로 변환
    new_df["mW/m2"] = (new_df["A(I열)"] * new_df["mV(F.C Ag/AgCl)"] * 1000) / 0.0007 * (-0.001)  # mW/m² 계산 후 -0.001로 수정

     # 빈 열 개수 설정
    num_empty_columns = 3  # 추가할 빈 열의 개수
    for i in range(num_empty_columns):
        new_df[f"빈열_{i+1}"] = np.nan  # 빈 열을 NaN으로 추가
    
    # 빈 열 이름을 공백으로 설정
    new_df.columns = ["" if "빈열" in col else col for col in new_df.columns]


    # H, I, J 열 추가
    new_df["기준전극 전압(V)(P열)"] = df["Auxiliary1_V"]  # 기준전극 전압(V)
    new_df["기준전극 전압(mV)(Anode Ag/AgCl)"] = df["Auxiliary1_V"] * 1000  # 기준전극 전압(mV)

    # 새로운 계산 추가
    new_df["Cathode potential (Cathode Ag/AgCl)"] = (df["Voltage_V"] * 1000) + (df["Auxiliary1_V"] * 1000)  # Cathode potential
    new_df["전극 보정(Anode) SHE"] = (df["Auxiliary1_V"] * 1000) + 209 - parameter  # 전극 보정
    new_df["cathode 값 SHE"] = (df["Voltage_V"] * 1000) + (df["Auxiliary1_V"]*1000 + 209 - parameter)  # cathode 값 SHE

else:
    raise ValueError("Auxiliary1_V 또는 Voltage_V 열이 원본 데이터에 없습니다. 열 이름을 확인하세요.")



# 데이터 확인 및 저장

In [8]:

# 3. 데이터 확인
print("정제된 데이터:\n", new_df.head())

# 4. CSV로 저장 (인코딩 설정)
new_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"정제된 데이터가 {output_csv}로 저장되었습니다.")

정제된 데이터:
          A(I열)     V(J열)  mV(F.C Ag/AgCl)       mA/m2       mW/m2              \
1799  0.000000  0.423582       423.581600    0.000000   -0.000000 NaN NaN NaN   
1800 -0.000476  0.422179       422.178745 -679.464720  286.855563 NaN NaN NaN   
1801 -0.000517  0.421215       421.215415 -737.983667  310.850097 NaN NaN NaN   
1802 -0.000551  0.420221       420.221001 -787.499677  330.923902 NaN NaN NaN   
1803 -0.000586  0.419227       419.226587 -837.015687  350.899229 NaN NaN NaN   

      기준전극 전압(V)(P열)  기준전극 전압(mV)(Anode Ag/AgCl)  \
1799       -0.546723                 -546.722889   
1800       -0.546882                 -546.882451   
1801       -0.546786                 -546.786070   
1802       -0.546459                 -546.459496   
1803       -0.546411                 -546.411335   

      Cathode potential (Cathode Ag/AgCl)  전극 보정(Anode) SHE  cathode 값 SHE  
1799                          -123.141289       -343.622889      79.958711  
1800                          -124.7